In [1]:
import pennylane as qml
from pennylane import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
import time
from pennylane.optimize import nesterov_momentum, gradient_descent, adam

np.set_printoptions(threshold=np.inf)

In [2]:
class Problem:
    def __init__(self, M, N, lam, d, in_angles, out_angles, constraints=None):
        self.M = M
        self.N = N
        self.lam = lam
        self.d = d
        self.theta_in = in_angles[0]
        self.phi_in = in_angles[1]
        self.theta_out = out_angles[0]
        self.phi_out = out_angles[1]
        self.k = 2 * np.pi / lam
        self.Q = None  # will be computed when needed
        self.constraints = constraints

    def phase(self, m, n, theta_out, phi_out):
        part1 = -m * np.sin(theta_out) * np.cos(phi_out)
        part2 = -n * np.sin(theta_out) * np.sin(phi_out)
        part3 = m * np.sin(self.theta_in) * np.cos(self.phi_in)
        part4 = n * np.sin(self.theta_in) * np.sin(self.phi_in)
        return self.k * self.d * (part1 + part2 + part3 + part4)

    def compute_Q(self, theta_out = None, theta_in = None):

      if theta_out is None:
        theta_out = self.theta_out
      if theta_in is not None:
        theta_in = self.theta_in

      Q = np.zeros((self.M * self.N, self.M * self.N))
      for row in range(self.M * self.N):
          m1, n1 = divmod(row, self.M)
          for col in range(row):
              m2, n2 = divmod(col, self.M)
              phase_diff = self.phase(m1, n1, theta_out, phi_out) - self.phase(m2, n2, theta_out, phi_out)
              Q[row, col] = np.cos(phase_diff)
          # Q[row, row] = 0
      self.Q = Q

      return Q

    def array_factor(self,  w_arr, Q = None):
      if Q is None:
        if self.Q is None:
            self.compute_Q()
        Q = self.Q
      arr_fac = (1 / (self.M * self.N) ** 2) * (1*M*N + w_arr.T @ Q @ w_arr)
      return arr_fac[0,0]

    def check_constraints(self, w_arr):
        if self.constraints is None:
            return "No constraints"

        checks = []
        for constraint in self.constraints:
          theta_o = constraint[0]
          phi_o = constraint[1]
          threshold = constraint[2]

          Q_i = self.compute_Q(theta_o, phi_o)
          arr_fac_i = self.array_factor(w_arr, Q = Q_i)

          if arr_fac_i <= threshold:
            checks.append(True)
          else:
            checks.append(False)

        return checks

def qaoa_circuit(gammas, betas, h, J, num_qubits):
    wmax = max(
        np.max(np.abs(list(h.values()))), np.max(np.abs(list(h.values())))
    )  # Normalizing the Hamiltonian is a good idea
    p = len(gammas)
    # Apply the initial layer of Hadamard gates to all qubits
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
    # repeat p layers the circuit shown in Fig. 1
    for layer in range(p):
        # ---------- COST HAMILTONIAN ----------
        for ki, v in h.items():  # single-qubit terms
            qml.RZ(2 * gammas[layer] * v / wmax, wires=ki[0])
        for kij, vij in J.items():  # two-qubit terms
            qml.CNOT(wires=[kij[0], kij[1]])
            qml.RZ(2 * gammas[layer] * vij / wmax, wires=kij[1])
            qml.CNOT(wires=[kij[0], kij[1]])
        # ---------- MIXER HAMILTONIAN ----------
        for i in range(num_qubits):
            qml.RX(-2 * betas[layer], wires=i)
    return qml.sample()


def split_params(params, p):
    return params[:p], params[p:]

def combine_params(gammas, betas):
    return np.concatenate([gammas, betas])






class QAOAQUBO:
    def __init__(self, Q, p=1, optimizer='COBYLA', steps=100, seed=42):
        self.Q = Q
        self.n_qubits = Q.shape[0]
        self.p = p  # Number of QAOA layers
        self.steps = steps
        self.seed = seed
        np.random.seed(seed)

        # Choose optimizer
        self.opt_name = optimizer.upper()
        self.opt = self._get_optimizer(self.opt_name)

        # Define device
        self.dev = qml.device('default.qubit', wires=self.n_qubits, shots=10000)

        # Construct cost Hamiltonian
        self.cost_h = self.qubo_cost_hamiltonian(Q)
        print("Hamiltonian wires:", self.cost_h.wires)

        # Define circuit and cost function
        self.qnode = qml.QNode(self.qaoa_circuit, self.dev)
        self.cost_fn = lambda params: self.qnode(params)


    # def qubo_cost_hamiltonian(self, Q):
    #     coeffs = []
    #     ops = []
    #     for i in range(self.n_qubits):
    #         coeffs.append(Q[i, i] / 2)
    #         ops.append(qml.PauliZ(i))
    #         for j in range(i+1, self.n_qubits):
    #             if Q[i, j] != 0:
    #                 coeffs.append(Q[i, j] / 4)
    #                 ops.append(qml.PauliZ(i) @ qml.PauliZ(j))
    #     return qml.Hamiltonian(coeffs, ops)

    '''
    Above implementation is useful only if Q is defined in a basis where w is binary. In our problem w is +-1 so Q matrix can naturally be taken to be in the Z basis.
    '''

    def qubo_cost_hamiltonian(self, Q):
        coeffs = []
        ops = []
        for i in range(self.n_qubits):
            if Q[i, i] != 0:
                coeffs.append(Q[i, i])
                ops.append(qml.PauliZ(i))
            for j in range(i + 1, self.n_qubits):
                if Q[i, j] != 0:
                    coeffs.append(Q[i, j])
                    ops.append(qml.PauliZ(i) @ qml.PauliZ(j))
        return qml.Hamiltonian(coeffs, ops)

    def qaoa_circuit(self, params):

        gammas, betas = split_params(params, self.p)

        for i in range(self.n_qubits):
            qml.Hadamard(i)

        for layer in range(self.p):
            # Apply cost unitary
            qml.ApproxTimeEvolution(self.cost_h, gammas[layer], 1)
            # Apply mixer unitary
            for i in range(self.n_qubits):
                qml.RX(2 * betas[layer], i)

        return qml.expval(self.cost_h)


    def _get_optimizer(self, name):
        opts = {
            'GD': gradient_descent.GradientDescentOptimizer,
            'NESTEROV': nesterov_momentum.NesterovMomentumOptimizer,
            'ADAM': adam.AdamOptimizer
        }
        if name not in opts:
            raise ValueError(f"Optimizer '{name}' not supported.")
        return opts[name](stepsize=0.1)


    def optimize(self):
        gammas = np.array(np.random.uniform(0, np.pi, self.p), requires_grad=True)
        betas = np.array(np.random.uniform(0, np.pi, self.p), requires_grad=True)
        params = combine_params(gammas, betas)

        print("Initial gammas:", gammas)
        print("Initial betas:", betas)

        for i in range(self.steps):
            params = self.opt.step(self.cost_fn, params)
            params = np.array(params, requires_grad=True)  # rewrap with gradient tracking
            if i % 10 == 0:
                print(f"Step {i}: Cost = {self.cost_fn(params):.6f}")

        final_cost = self.cost_fn(params)
        return split_params(params, self.p), final_cost



    def get_best_bitstring(self, gammas, betas):
        # @qml.qnode(self.dev)
        # def sample_circuit():
        #     for i in range(self.n_qubits):
        #         qml.Hadamard(i)
        #     for layer in range(self.p):
        #         qml.ApproxTimeEvolution(self.cost_h, gammas[layer], 1)
        #         for i in range(self.n_qubits):
        #             qml.RX(2 * betas[layer], i)
        #     return qml.sample(wires=range(self.n_qubits))

        # samples = np.array([sample_circuit() for _ in range(num_samples)])

        @qml.qnode(self.dev)
        def sample_circuit():
            for i in range(self.n_qubits):
                qml.Hadamard(i)
            for layer in range(self.p):
                qml.ApproxTimeEvolution(self.cost_h, gammas[layer], 1)
                for i in range(self.n_qubits):
                    qml.RX(2 * betas[layer], i)
            return qml.sample(wires=range(self.n_qubits))

        samples = sample_circuit()

        best_score = -np.inf

        best_sample = None
        for sample in samples:
            w = 1-2 * sample.reshape(-1, 1)  #because measurement returns 0 or 1, we convert to -1 or 1
            score = float((w.T @ self.Q @ w)[0, 0])
            if score > best_score:
                best_score = score
                best_sample = w.flatten()  # now we've converted result to now in {−1, +1}
        return best_sample, best_score


In [ ]:

### We define the parameters of our problem and initialize them in the Problem class
M = 3
N = 3
lam = 1e-3
d = lam / 1


# theta_in = np.pi / 4
# phi_in = np.pi / 3
# theta_out = np.pi / 3
# phi_out = np.pi / 4


theta_in = np.pi / 4
phi_in = np.pi / 3
theta_out = theta_in
phi_out = np.pi - phi_in

# For usual reflection problems, theta_in = theta_out and phi_in = pi - phi_out so we've introduced the same case here. 


prob = Problem(
    M, N, lam, d,
    in_angles=(theta_in, phi_in),
    out_angles=(theta_out, phi_out)
)

# From the problem we compute the lower-triangular Q matrix (corresponding to quadratic terms in |G^2| calculation) after which qubo matrix is computed as Q_input 

Q = prob.compute_Q()
Q_input = Q + Q.T

qaoa = QAOAQUBO(Q_input, p=2, optimizer='ADAM', steps=100) 


Hamiltonian wires: Wires([0, 1, 2, 3, 4, 5, 6, 7, 8])


### QAOA: First we get the optimal circuit parameters ($\gamma$ for trotter-evolving cost hamiltonian, $\beta$ for mix hamiltonian)

In [4]:
(opt_gammas, opt_betas), final_cost = qaoa.optimize()

Initial gammas: [0.57627739 2.44947152]
Initial betas: [1.87506007 1.4006249 ]
Step 0: Cost = 1.863777
Step 10: Cost = -0.384274
Step 20: Cost = -2.614944
Step 30: Cost = -2.970108
Step 40: Cost = -3.082428
Step 50: Cost = -3.174650
Step 60: Cost = -3.208683
Step 70: Cost = -3.211481
Step 80: Cost = -3.183982
Step 90: Cost = -3.182327


### Once we get the optimal params, we evolve the state in multiple shots and derive the bitstring, from which the best value is stored. This corresponds to the optimal [w] array

In [ ]:
print("Optimal gammas:", opt_gammas)
print("Optimal betas:", opt_betas)
print("Final cost (QAOA):", final_cost)

best_w, best_score = qaoa.get_best_bitstring(opt_gammas, opt_betas)

print("\n\n\nBest omega (bitstring):", best_w)
print("\nMaximized cost (wᵗQw):", best_score)

w_arr = best_w.reshape(-1, 1)
constraint_results = prob.check_constraints(w_arr)

# Final array factor
af = prob.array_factor(w_arr, Q_input)

# Convert AF to decibels, avoiding log of zero
af_clipped = max(af, 1e-10)
af_dB = 10 * np.log10(af_clipped)
print(f"\nFinal Array Factor (AF): {af:.6f}")
print(f"AF in dB: {af_dB:.2f} dB")


print("\n\n\nConstraint satisfaction:")
if constraint_results == "No constraints":
    print("No constraints defined.")
else:
    for i, result in enumerate(constraint_results):
        print(f"  Constraint {i+1}: {'✅ Passed' if result else '❌ Failed'}")



Optimal gammas: [0.21432444 2.15603732]
Optimal betas: [1.2935722  1.61927585]
Final cost (QAOA): -3.1850966236032776



Best omega (bitstring): [-1 -1 -1  1  1  1  1  1  1]

Maximized cost (wᵗQw): 33.44789134203872

Final Array Factor (AF): 0.524048
AF in dB: -2.81 dB



Constraint satisfaction:
No constraints defined.


### Below we do a brute force search for the best solution:

In [6]:
def cost(x,Q):
    return x.T @ Q @ x

from itertools import product
  # total length = M * N

cost_optimal = 0
cur_cost =0
for x in product([-1, 1], repeat=M * N):
    # print(np.array(x))
    cur_cost = cost(np.array(x),Q_input)
    if(cur_cost>cost_optimal):
        cost_optimal = cur_cost
        w_array_optimal = x

print(cost_optimal)
print(w_array_optimal)

33.447891342038744
(-1, -1, -1, -1, -1, -1, 1, 1, 1)


We find that the answers match perfectly; This was in fact tested over a various cases of angles and gives the right result every time. (For the unconstrianed problem)

### For the constrained problem, we do as below:




In [ ]:

### We define the parameters of our problem and initialize them in the Problem class
M = 3
N = 3
lam = 1e-3
d = lam / 1


# theta_in = np.pi / 4
# phi_in = np.pi / 3
# theta_out = np.pi / 3
# phi_out = np.pi / 4


Q = prob.compute_Q()
Q_input = Q + Q.T

theta_in = np.pi / 4
phi_in = np.pi / 3
theta_out = theta_in
phi_out = np.pi - phi_in

# For usual reflection problems, theta_in = theta_out and phi_in = pi - phi_out so we've introduced the same case here. 

prob = Problem(
    M, N, lam, d,
    in_angles=(theta_in, phi_in),
    out_angles=(theta_out, phi_out)
)

constraints = [(np.pi/6, np.pi/6, 3)] #initialized to some random values; the idea is if theta, phi lies here then it is penalized in the cost function
prob.constraints = constraints


Q_penalty =Problem(
    M, N, lam, d,
    in_angles=(theta_in, phi_in),
    out_angles=(np.pi/6, np.pi/6)
).compute_Q()

Q_input_penalty = Q_penalty + Q_penalty.T

l = 5 #hyperparameter for penalty term; the larger the value, the more the penalty is applied to the cost function
Q_input_tot = Q_input - l* Q_input_penalty

qaoa = QAOAQUBO(Q_input, p=2, optimizer='ADAM', steps=100) 


(opt_gammas, opt_betas), final_cost = qaoa.optimize()
print("Optimal gammas:", opt_gammas)
print("Optimal betas:", opt_betas)
print("Final cost (QAOA):", final_cost)

best_w, best_score = qaoa.get_best_bitstring(opt_gammas, opt_betas)

print("\n\n\nBest omega (bitstring):", best_w)
print("\nMaximized cost (wᵗQw):", best_score)

w_arr = best_w.reshape(-1, 1)
constraint_results = prob.check_constraints(w_arr)

# Final array factor
af = prob.array_factor(w_arr, Q_input)

# Convert AF to decibels, avoiding log of zero
af_clipped = max(af, 1e-10)
af_dB = 10 * np.log10(af_clipped)
print(f"\nFinal Array Factor (AF): {af:.6f}")
print(f"AF in dB: {af_dB:.2f} dB")


print("\n\n\nConstraint satisfaction:")
if constraint_results == "No constraints":
    print("No constraints defined.")
else:
    for i, result in enumerate(constraint_results):
        print(f"  Constraint {i+1}: {'✅ Passed' if result else '❌ Failed'}")

